In [1068]:
import pandas as pd
import numpy as np
import math

# I. Numereisation

## 1. Recuperation de donnee

In [1069]:
def load_data_from_csv(csv_path:str):
    data = pd.read_csv(csv_path)
    return data

## 2. Encodage

In [1070]:
def encodage(data,indexes:list,encode_rules):
    ''' Permet de faire l'encodage des donnees 
        ARGS
            - data : la liste des donnees
            - indexes : la listes des colonnes a encoder
            - encode_rules : un dictionaire des regles a suivre pour encodes la valeur des donnees
    '''
    for i in range(len(data)):
        for index in indexes:
            value = data.loc[i,index]
            try:
                value = encode_rules[index][value]
            except :
                value = encode_rules[value]
            data.loc[i,index] = value

# II. Entropie

In [1071]:
def entopie_piece(repetion:int,total:int):
    result = repetion / total
    return result

In [1072]:
def entropie(data:pd.DataFrame,class_attribute:str):
    ''' Etablir l'entropie a partir d'une colonne de classe 
        PARAMETERS :
            - data : les donnees pour etablir l'entropie
            - class_attribute : Le nom de la colonne qui represente les donnee pour l'entropie
        RETURN :
            - entropie , la valeur entropique des donnees par rapport au colonne donnee
    '''
    # Le nombre total de donnee
    data_lenght = len(data)
    # Recuperer les valeurs de la classe
    class_values = data[class_attribute]
    # Recupere en unique les valeur de la class
    class_unique_values = set(class_values)
    # Calculer la repetions de chacunes des valeurs parmi les donnees
    repetitions = {
        valeur: class_values.tolist().count(valeur) 
        for valeur in class_unique_values
    }
    # Le resultat attendue
    entropie = 0
    # Calcul de l'entropie
    for value in class_unique_values:
        # La piece entropique de la valeur
        pc = entopie_piece(repetitions[value],data_lenght)
        # Incrementation de la somme
        it = -(pc) * math.log2(pc)
        entropie += it
    return entropie

# III. Meilleure Caracteristique

## 1. Gain

### 1.1. Separation de donnee

In [1073]:
def split_data(dataframe,colonne):
    """
    Sépare les données d'un DataFrame selon une colonne spécifique en regroupant les mêmes critères.

    Parameters:
    dataframe (pd.DataFrame): Le DataFrame contenant les données.
    colonne (str): Le nom de la colonne selon laquelle séparer les données.

    Returns:
    dict: Un dictionnaire où les clés sont les valeurs uniques de la colonne et les valeurs sont les DataFrames correspondants.
    """
    groupes = dataframe.groupby(colonne)
    resultats = {nom: groupe for nom , groupe in groupes}
    return resultats

### 1.2. Fonction de gain

In [1074]:
def gain(data:pd.DataFrame,ref_attribute:str,class_attribute:str):
    # Entropie de l'ensemble du data
    entropie_S = entropie(data,class_attribute)
    # Le gain attendue
    gain = entropie_S
    # Le nombre de donnee total
    nS = len(data)
    # Separation des donnees en plusieurs sous section de donnee
    data_sv = split_data(data,ref_attribute)
    for index, data_part in data_sv.items():
        # Le nombre de sous section
        nSv = len(data_part)
        sv = nSv/nS
        # Entropie du sous section
        entropie_SV = entropie(data_part,class_attribute)

        gain -= sv*entropie_SV
    return gain

## 2. Calculs des GAINS

In [1075]:
def gains(data:pd.DataFrame,class_attribute:str):
    ''' 
        Calcul des gains pour chaque critere present parmi les donnees
        ARGS :
            - data : la liste des donnees d'entrainement
            - class_attribute : le nom de la colonne qui prescise la class des donees
        RETURNS :
            - gains , tableau des couples ( nom critere , gain )
    '''
    # Resulats > Liste des gains
    gains = list()
    # Recuperation des criteres possibles
    indexes = data.columns
    for index in indexes :
        if index == class_attribute:
            continue
        gains.append({index : gain(data,index,class_attribute)})
    return gains

## 3. Best collumn

In [1076]:
def find_best_criteria(data:pd.DataFrame,class_attribute:str):
    '''
        Calcul le gain d'information de chaque critere de donnee pour determiner le meilleur parmi eux
        ARGS :
            - data : la liste des donnees d'entrainement
            - class_attribute : le nom de la colonne qui prescise la class des donees
        RETURNS :
            - criteria , le nom du critere qui possede le gain la plus haute apres filtre decroissante
    '''
    # Recuperation des gains pour chaque criteres
    gains_var = gains(data,class_attribute)
    # Filtrer decroissant
    gains_sorted = sorted(gains_var, key=lambda d : list(d.values())[0])
    gains_sorted.reverse()
    return [criteria for criteria , gain_value in gains_sorted[0].items()][0]

# IV. ARBRE

In [1077]:
def generate_tree(data:pd.DataFrame,class_attribute:str):
    # The tree decision
    tree = dict()
    # Recuperer le meilleur critre
    best_criteria = find_best_criteria(data,class_attribute)
    # Diviser en sous section des donnees du best criteria
    splited_data = split_data(data,best_criteria)
    branch = {}
    for key in splited_data.keys():
        # Data de chaque section
        data_part = splited_data[key]
        # On verifie l'entropie 
        data_entropie = entropie(data_part,class_attribute)
        if data_entropie == 0:
            # Pour une entropie homogene
            data_class = np.array(data_part[class_attribute])[0]
            branch[key] = data_class
        else :
            branch[key] = generate_tree(data_part,class_attribute)
        tree[best_criteria] = branch
    return tree

# Main

In [1078]:
# Recuperation des donnees
data = load_data_from_csv("data/spam.csv")
# Encodage des donnees
# encodage(
#     data,
#     indexes=['Previsions','Temperature','Humidite','Vent','Classe'],
#     encode_rules={
#         'Ensoleille':1,'Nuageux':2,'Pluvieux':3,
#         'Chaud':3,'Moyen':2,'Frais':1,
#         'Elevee':2,'Normale':1,
#         'Faible':1,'Fort':2,
#         'Oui':0,'Non':1
#     }
# )

# Creation d'un arbre
tree = generate_tree(data,'Classe')

Ensoleille


In [1079]:
{'Previsions': 
    {'Ensoleille': 
        {'Humidite': 
            {
                'Elevee': 'Non',
                'Normale': 'Oui'
            }
        },
  'Nuageux': 'Oui',
  'Pluvieux': 
        {
      'Vent': 
            {
          'Faible': 'Oui', 'Fort': 'Non'
            }
        }
    }
}

{'Previsions': {'Ensoleille': {'Humidite': {'Elevee': 'Non',
    'Normale': 'Oui'}},
  'Nuageux': 'Oui',
  'Pluvieux': {'Vent': {'Faible': 'Oui', 'Fort': 'Non'}}}}